In [15]:
import os
import sys
import numpy as np
import torch
import plotly.graph_objects as go
import tqdm
import pickle
import kaleido
import astropy.units as u
from astroquery.ipac.irsa import Irsa
ROOT = os.path.join("./")
sys.path.append(ROOT + "lib")
from helpers import get_coordinates
from lightsource import LightSource
from plotly_style import update_layout

buckets = pickle.load(open(ROOT + "cached_data/raw_classes.pkl", "rb"))

In [18]:
def get_trace(lightsource, key):

  x_data = lightsource["norm"]["day"]
  y_data = lightsource["norm"][key]

  sort_idxs = np.argsort(x_data)

  x_data = x_data[sort_idxs]
  y_data = y_data[sort_idxs]


  return go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(size=5, opacity=.75),
    name=key,
  )

def plot_from_lightsource(lightsource, ys=["w1"]):

  fig = go.Figure()

  for y in ys:
    fig.add_trace(get_trace(lightsource, y))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width

  return fig

def plot_from_tensor(data):
  fig = go.Figure()

  w1 = data[:, 0].numpy()
  std = data[:, 1].numpy()


  day = data[:, -1].numpy()


  fig.add_trace(go.Scatter(x=day, y=w1, marker=dict(size=5, opacity=0.7), name="w1mpro z-scored", mode='markers'))
  fig.add_trace(go.Scatter(x=day, y=std, marker=dict(size=5, opacity=0.7), name="w2mpro z-scored", mode='markers'))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width


  return fig

In [20]:
# ----------------------- Begin Work on Dataset ----------------------- # 
ex = buckets["nova"]["V569 Vul"]
flux = ex["raw"]["w1flux"]

ADJ = 0
DIV = 0.001
norm = np.arcsinh((flux - ADJ) /DIV)
tr1 = go.Scatter(
    x=ex["norm"]["day"],
    y=norm,
    marker=dict(size=5, opacity=0.7), name="normalized flux", mode='markers'
)

tr2 = go.Scatter(
    x=ex["norm"]["day"],
    y=flux,
    marker=dict(size=5, opacity=0.7), name="raw flux", mode='markers'
)

fig = plot_from_lightsource(ex, ["w1"])
fig.add_trace(tr1)
fig.add_trace(tr2)
fig.show()

print([n for n in flux])


[0.05034041346544911, 0.04960400907233546, 0.05089988232039497, 0.05222960943443111, 0.0615909323305002, 0.06302556944183903, 0.060022872879444036, 0.05726862793792811, 0.05611988575584492, 0.05408997469394473, 0.05550304094614799, 0.007176567570141802, 0.006961723626904082, 0.0066913967066320095, 0.006563217221556538, 0.007283108472479406, 0.006878864871647522, 0.0068095243665510395, 0.007117324683565499, 0.006936122876513555, 0.00696813856505673, 0.007398041756465824, 0.006759534333059773, 0.007229641766603696, 0.0074048587435888175, 0.007696916555218579, 0.003643483257982175, 0.003409704778754495, 0.003663673627218743, 0.0034891270058456793, 0.0031909263347898484, 0.003447599255516844, 0.003453955820854343, 0.0032115654583230765, 0.0032954655671986292, 0.0037593847750054268, 0.0035671122689832947, 0.0033846734774608915]


In [21]:
c = get_coordinates("19 52 08.25	+27 42 20.9")

tbl = Irsa.query_region(c, catalog="neowiser_p1bs_psd", spatial="Cone",
                            radius=5 * u.arcsec)
tbl = tbl.to_pandas()

tbl = tbl.loc[tbl["qual_frame"]>= 4]
ex = LightSource(tbl, "")
plot_from_lightsource(ex)


In [48]:
to_flux_w1 = lambda m: 309.54 * 10**(-m / 2.5)
tr = go.Scatter(
    x=tbl["ra"],
    y=tbl["dec"],
    marker=dict(size=np.log(10**6*to_flux_w1(tbl["w1mpro"])) / 1.4, opacity=0.4, color="red"), name="pos", mode='markers',
)

fig = go.Figure()
fig.add_trace(tr)
fig.show()